In [152]:
import numpy as np
import pandas as pd
from datetime import datetime

In [120]:
rd_station_profile = pd.read_csv("../data/rd_station_profile")
info_student_unimestre = pd.read_csv("../data/info_student_unimestre.csv")

### Checking the data frames


In [121]:
rd_station_profile.shape

(41340, 32)

In [122]:
info_student_unimestre.shape

(12056, 5)

In [123]:
info_student_unimestre.sample()

,cd_pessoa,ds_email,ds_cidade,dt_nascimento,min_dt_log
10486,160004572,josiasbio@bol.com.br,NaN,1979-12-23 00:00:00,NaN


In [170]:
info_student_unimestre.isna().sum()

cd_pessoa           0
ds_email            0
ds_cidade        2248
dt_nascimento    1156
min_dt_log       8144
dtype: int64

### Checking how many rows got in a inner join (getting just the intersection)

In [124]:
merge_inner = rd_station_profile.merge(info_student_unimestre, left_on='email', right_on='ds_email', how='inner')

In [125]:
merge_inner.shape

(4150, 37)

### Left Join

In [126]:
merge_left = rd_station_profile.merge(info_student_unimestre, left_on='email', right_on='ds_email', how='left')

In [127]:
merge_left.shape # 41428 got more rows than the origial

(41428, 37)

Got more rows than the left data frame, so will check for duplicates values 

In [128]:
rd_station_profile.groupby("email")["email"].count()

email
0137.marcogiovanella@cnec.br     1
0337.mauro@bradesco.com.br       1
0337.william@bradesco.com.br     1
0358.camila@bradesco.com.br      1
0358.gerencia@bradesco.com.br    1
                                ..
zullo_gui@hotmail.com            1
zulmafernandesstolf@gmail.com    1
zulmalira.contabil@gmail.com     1
zwessel@hotmail.com              1
zyon.beto@gmail.com              1
Name: email, Length: 41340, dtype: int64

In [131]:
info_student_unimestre.groupby("ds_email")["ds_email"].count()

ds_email
 leory@r19.com.br                     1
 renatomontag@terra.com.br            1
00000000/00@sememail.com             27
00000000000@sememail.com            210
00000000001@sememail.com              1
                                   ... 
znf.nem@gmail.com                     1
zpierdona@prsp.mpf.gov.br             1
zuleikaromao@gomesdacosta.com.br      1
zulmarpr@gmail.com                    1
zvirtes@joinville.udesc.br            1
Name: ds_email, Length: 10520, dtype: int64

In [130]:
info_student_unimestre.groupby("cd_pessoa")["cd_pessoa"].count()

cd_pessoa
1001         1
1002         1
100003       1
100011       1
100046       1
            ..
170000461    1
170000462    1
170000463    1
170000464    1
170000465    1
Name: cd_pessoa, Length: 12056, dtype: int64

Creating a df to be only one row per ds_email

In [132]:
info_student_unimestre_unique = info_student_unimestre.groupby('ds_email').first().reset_index()

In [133]:
info_student_unimestre_unique.shape

(10520, 5)

### Doing the join again with the new df

In [134]:
merge_left = rd_station_profile.merge(info_student_unimestre_unique, left_on='email', right_on='ds_email', how='left')

In [135]:
merge_left.shape

(41340, 37)

In [136]:
merge_left = merge_left.drop(columns='Unnamed: 0')

In [137]:
merge_left.columns

Index(['email', 'lead_scoring_-_perfil', 'url_pública', 'estágio_no_funil',
       'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'data_de_nascimento', 'desejo_receber_o_sustentare_news',
       'como_você_conheceu_a_sustentare:', 'universidade',
       'qual_o_curso_de_interesse?', 'cidade_final', 'cargo_final',
      

Creating a new column saying it's a student when a cd_pessoa is found

In [138]:
merge_left['aluno'] = (merge_left['cd_pessoa'].notnull()).astype(int)
merge_left['aluno'].value_counts()


aluno
0    37278
1     4062
Name: count, dtype: int64

## Got different columns with representing the same thing, compiling all in one new columns

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [139]:
merge_left.loc[:, 'cidade'] = (
    merge_left['ds_cidade']
    .fillna(merge_left['cidade_final'])
)

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [140]:
merge_left.loc[:, 'data_nascimento'] = (
    merge_left['dt_nascimento']
    .fillna(merge_left['data_de_nascimento'])
)

In [141]:
merge_left.isna().sum()

email                                                                    0
lead_scoring_-_perfil                                                    0
url_pública                                                              0
estágio_no_funil                                                         0
total_de_conversões                                                      0
lead_scoring_-_interesse                                                 0
status_para_comunicação_por_email                                        0
data_da_primeira_conversão                                               0
eventos_(últimos_100)                                                    0
origem_da_última_conversão                                               0
data_da_última_conversão                                                 0
origem_da_primeira_conversão                                             0
empresa                                                              23786
tags                     

With this I was able to reduce 
- *Cidade* had 35856 null values now 33986
- *Data Nascimento* had 38346 null values now 36117

Dropping columns with replicate values

In [142]:
merge_left = merge_left.drop(columns=['ds_cidade', 'cidade_final', 'dt_nascimento', 'data_de_nascimento'])

All not null

In [143]:
merge_left[merge_left.notna().all(axis=1)]

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,qual_o_curso_de_interesse?,cargo_final,interesse_final,area_atuacao,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento
5431,carolineklock@hotmail.com,c,http://app.rdstation.com.br/leads/public/8717b...,Lead Qualificado,16,120,True,2015-09-01 19:17:37 -0300,jornada-assessment-instrumentos-para-lideranca...,Busca Orgânica | Bing,...,Educação empresarial / Remuneração,Outros Cargos,formulario-de-pre-inscricao,RH,carolineklock@hotmail.com,128609.0,2018-04-05 19:53:30,1,Joinville,1985-09-21 00:00:00
15881,stella.bousfield@gmail.com,c,http://app.rdstation.com.br/leads/public/1b271...,Cliente,27,170,True,2015-12-08 10:18:55 -0200,webinar-storytelling / indicado-sarau-de-negoc...,Desconhecido,...,Responsabilidade social,Outros Cargos,webinar-storytelling,Marketing,stella.bousfield@gmail.com,160000017.0,2020-02-27 17:28:20,1,Joinville,1977-06-02 00:00:00


### Change dtype for columns that are date

In [163]:
date_columns = ['data_da_primeira_conversão', 'data_da_última_conversão', 'data_da_última_oportunidade', 'data_nascimento', 'min_dt_log']

# Apply pd.to_datetime to each column and then remove timezone information
merge_left[date_columns] = merge_left[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce').dt.tz_localize(None))


In [166]:
def convert_columns_to_datetime(df, date_columns):
    """
    Convert specific columns in a DataFrame to datetime format and remove timezone information.

    Parameters:
    - df: DataFrame
        The input DataFrame.
    - date_columns: list
        List of column names to convert to datetime.

    Returns:
    - DataFrame
        The DataFrame with specified columns converted to datetime and timezone information removed.
    """
    df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce').dt.tz_localize(None))
    return df

# Example usage:
date_columns = ['data_da_primeira_conversão', 'data_da_última_conversão', 'data_da_última_oportunidade', 'data_nascimento', 'min_dt_log']
merge_left = convert_columns_to_datetime(merge_left, date_columns)


In [167]:
merge_left.dtypes

email                                                                        object
lead_scoring_-_perfil                                                        object
url_pública                                                                  object
estágio_no_funil                                                             object
total_de_conversões                                                           int64
lead_scoring_-_interesse                                                      int64
status_para_comunicação_por_email                                              bool
data_da_primeira_conversão                                           datetime64[ns]
eventos_(últimos_100)                                                        object
origem_da_última_conversão                                                   object
data_da_última_conversão                                             datetime64[ns]
origem_da_primeira_conversão                                                

In [169]:
def calculate_months_since_conversion(df):
    """
    Calculates the number of months since the first conversion based on the 'aluno' column.

    Parameters:
    - The input DataFrame containing the necessary columns.

    Returns:
    - A new column representing the number of months since the first conversion.
    """

    # If 'aluno' is 1, calculate the difference between 'min_dt_log' and 'data_da_primeira_conversão'
    # If not calculate the difference between today and 'data_da_primeira_conversão'
    months_since_conversion = np.where(
        df['aluno'] == 1,
        (df['min_dt_log'] - df['data_da_primeira_conversão']).dt.days / 30,
        (datetime.now() - df['data_da_primeira_conversão']).dt.days / 30
    )

    return months_since_conversion

merge_left['months_since_conversion'] = calculate_months_since_conversion(merge_left)

merge_left


,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,cargo_final,interesse_final,area_atuacao,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento,months_since_conversion
0,andersondorner30@gmail.com,d,http://app.rdstation.com.br/leads/public/d5637...,Lead,4,0,True,2015-03-19 03:00:00,whatsapp-button / whatsapp-button / Matriculad...,Busca Orgânica | Google,...,NaN,NaN,NaN,andersondorner30@gmail.com,108039.0,2015-03-19 21:33:34,1,Rio do sul,1982-06-09,0.000000
1,larissasuppi.op@gmail.com,c,http://app.rdstation.com.br/leads/public/c00b9...,Lead,17,70,True,2020-09-22 20:49:53,webinar-teoria-u-e-lideranca-humanizada / live...,Email | webinar_teoria_u_e_lideranca_humanizada,...,Coordenador/Líder,workshop-agile-marketing-growth-vendas-canais-...,Comercial,larissasuppi.op@gmail.com,160782.0,NaT,1,Jaraguá do Sul,1983-07-08,NaN
2,xavierleticia2016@gmail.com,d,http://app.rdstation.com.br/leads/public/60b83...,Lead,2,0,True,2020-05-04 22:25:57,webinar-deep-tech-e-a-organizacao-amplificada ...,Email | duplicado_de_webinar__deep_tech_e_a_or...,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,43.466667
3,qmxumc.cpjtqd@tonetics.biz,d,http://app.rdstation.com.br/leads/public/2e333...,Lead,1,0,True,2023-11-27 23:18:23,detalhes-do-curso,Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,0.066667
4,jcttud.cpjtjc@tonetics.biz,d,http://app.rdstation.com.br/leads/public/c517e...,Lead,1,0,True,2023-11-27 23:18:17,detalhes-do-curso,Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41335,planejamento@bauma.com.br,d,http://app.rdstation.com.br/leads/public/35e3c...,Lead,1,0,False,2014-08-16 03:00:00,"Exportação Mail2Easy - Grupos Projetos, TIC, T...",Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,113.100000
41336,roma@construtoraroma.com.br,d,http://app.rdstation.com.br/leads/public/75b62...,Lead,1,0,True,2014-08-16 03:00:00,"Exportação Mail2Easy - Grupos Projetos, TIC, T...",Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,113.100000
41337,financeiro@impressoraipiranga.com.br,d,http://app.rdstation.com.br/leads/public/62f26...,Lead,1,0,True,2014-08-16 03:00:00,"Exportação Mail2Easy - Grupos Projetos, TIC, T...",Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,113.100000
41338,compras@berlimambientes.com.br,d,http://app.rdstation.com.br/leads/public/fa19c...,Lead,1,0,True,2014-08-16 03:00:00,"Exportação Mail2Easy - Grupos Projetos, TIC, T...",Desconhecido,...,NaN,NaN,NaN,NaN,NaN,NaT,0,NaN,NaT,113.100000


In [159]:
def calculate_months_since_conversion(df):
    """
    Calculates the number of months since the first conversion based on the 'aluno' column.

    Parameters:
    - df: pandas DataFrame
        The input DataFrame containing the necessary columns.

    Returns:
    - pandas Series
        A new column representing the number of months since the first conversion.
    """
    # Ensure 'data_da_primeira_conversão' is a datetime type
    df['data_da_primeira_conversão'] = pd.to_datetime(df['data_da_primeira_conversão'])

    # If 'aluno' is 1, calculate the difference between 'min_dt_log' and 'data_da_primeira_conversão'
    # If not, calculate the difference between today and 'data_da_primeira_conversão'
    months_since_conversion = np.where(
        df['aluno'] == 1,
        (df['min_dt_log'] - df['data_da_primeira_conversão']).dt.days / 30,
        (datetime.now() - df['data_da_primeira_conversão']).dt.days / 30
    )

    return months_since_conversion

# Example usage:
# Assuming your DataFrame is named 'merge_left'
merge_left['months_since_conversion'] = calculate_months_since_conversion(merge_left)

# Display the DataFrame
print(merge_left)


TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects